# Visualization + dashboard

## Import packages

In [39]:
import carto2gpd
import geopandas as gpd
import pandas as pd
import numpy as np

import holoviews as hv

import hvplot.pandas

import esri2gpd
import seaborn as sns

from io import StringIO
from bokeh.models import HoverTool

import geoviews
import cartopy

from holoviews.element.tiles import CartoLight
import colorcet
import panel as pn
pn.extension()

import param

In [40]:
#Read files remotely
df = gpd.read_file('https://raw.githubusercontent.com/lizard12995/autohazard_explorer/main/upd_li_props.geojson').to_crs(epsg=3857)
philly_tracts = gpd.read_file('https://raw.githubusercontent.com/lizard12995/autohazard_explorer/main/philly_census.geojson').to_crs(epsg=3857)

In [41]:
import cenpy

acs = cenpy.remote.APIConnection("ACSDT5Y2020")

variables = [
    "NAME",
    "B03002_001E", # Total
    "B03002_003E", # Not Hispanic, White
    "B03002_004E", # Not Hispanic, Black
    "B03002_005E", # Not Hispanic, American Indian
    "B03002_006E", # Not Hispanic, Asian
    "B03002_007E", # Not Hispanic, Native Hawaiian
    "B03002_008E", # Not Hispanic, Other
    "B03002_009E", # Not Hispanic, Two or More Races
    "B03002_012E", # Hispanic
    "B19013_001E", # Median HH Income
]

philly_county_code = "101"
pa_state_code = "42"

phil_data = acs.query(
    cols=variables,
    geo_unit="tract:*",
    geo_filter={"state": pa_state_code, 
                "county": philly_county_code},
)

#Make integer

for variable in variables:
    
    # Convert all variables EXCEPT for NAME
    if (variable != "NAME") & (variable !="tract"):
        phil_data[variable] = phil_data[variable].astype(float)
        
phil_data = phil_data.rename(columns={
    "B03002_001E": "Total Population",
    "B03002_003E": "Not Hispanic, White",# 
    "B03002_004E": "Not Hispanic, Black", # 
    "B03002_005E": "Not Hispanic, American Indian", # 
    "B03002_006E": "Not Hispanic, Asian", # 
    "B03002_007E": "Not Hispanic, Native Hawaiian", # 
    "B03002_008E": "Not Hispanic, Other", # 
    "B03002_009E": "Not Hispanic, Two or More Races", # 
    "B03002_012E": "Hispanic", # 
    "B19013_001E": "Median HH Income"})

### Spatial join with census geometry to get tract info

In [42]:
#Join with Census geographies
df_tract = gpd.sjoin(df, philly_tracts, predicate='within',how='right')

#Sum values
tract_LI_data2 = df_tract.groupby("TRACT").sum().drop(columns="perc_311")

#Mean times
tract_LI_data3 = df_tract.groupby("TRACT")[["mean_time_to_investigation","time_to_resolution"]].mean()

#Mean percent 311
tract_LI_data4 = df_tract.groupby("TRACT")["perc_311"].mean()

tract_LI_data3 = tract_LI_data3.merge(tract_LI_data4, left_index=True,right_index=True)

#Merge totals with mean time data
tract_LI_data = tract_LI_data2.merge(tract_LI_data3, left_index=True,right_index=True).reset_index().drop(columns=["index_left"])

#Merge with Census data
tract_LI_data = tract_LI_data.merge(phil_data, how='left', left_on="TRACT",right_on="tract")

#Merge with Geometry
tract_LI_data = tract_LI_data.merge(philly_tracts)
tract_LI_data["Total Population_p"] = np.where(tract_LI_data["Total Population"]==0, 100000, tract_LI_data["Total Population"])

#Make it a geodataframe and set EPSG
tract_LI_data = gpd.GeoDataFrame(tract_LI_data, crs="EPSG:3857").to_crs(epsg=4326)

In [43]:
# Normalize variables
for i in ["Active","number_complaints","overall_num_investigations","total_violations"]:
    name = i + " Per 1000 People"
    tract_LI_data[name] = (tract_LI_data[i] / tract_LI_data["Total Population_p"])*1000

#Population percentages
for i in ['Not Hispanic, White',
       'Not Hispanic, Black', 'Not Hispanic, American Indian',
       'Not Hispanic, Asian', 'Not Hispanic, Native Hawaiian',
       'Not Hispanic, Other', 'Not Hispanic, Two or More Races', 'Hispanic']:
    name = "Percent " + i
    tract_LI_data[name] = round(((tract_LI_data[i] / tract_LI_data["Total Population"])*100),2)

In [44]:
#Drop fake population column
tract_LI_data = tract_LI_data.drop(columns=["Total Population_p", "state","county","tract",'NAME'])

## Long version
long_tract_LI_data = tract_LI_data.melt(id_vars = ["TRACT","geometry"])

In [45]:
#Other thing to fix
df["License"] = np.where(df["Active_TF"] == "0", False, True)

### Look at correlations

In [46]:
# vals = ["Active",'Expired','number_complaints','total_violations',"no_resolution",
#         'Percent Not Hispanic, White',
#        'Percent Not Hispanic, Black', 
#        'Percent Not Hispanic, Asian', 
#        'Percent Not Hispanic, Two or More Races', 'Percent Hispanic','Median HH Income']

# numeric = tract_LI_data[vals] 
# numeric = numeric.loc[numeric["Median HH Income"]>0]
# corr_matrix=numeric.corr()
# mask = np.zeros_like(corr_matrix)
# mask[np.triu_indices_from(mask)]=True

# sns.heatmap(corr_matrix, annot=True,fmt=".1f",mask=mask)

## Panel

In [47]:
#Tab 1 map

def tract_map(Variable = 'Active', Minimum_Population = 0):
    return CartoLight()*tract_LI_data.loc[tract_LI_data['Total Population']>=Minimum_Population].hvplot(
        c=Variable,
                     geo=True,
                     legend=True,
                                             #clim=clim,
                     width=300,
                     height=200,
                     cmap='viridis',
                    title='L&I and Demographic Data by Census Tract', 
                     frame_height=800,
                     frame_width=800,
                                  alpha = .5,
                                  line_alpha=.8,
                                  xaxis=None, yaxis=None,
                     hover_cols=['TRACT', 'Total Population','Percent Not Hispanic, White',
                                'Percent Hispanic', 'Percent Not Hispanic, Black',
                                'Active','number_complaints','total_violations'])


columns=tract_LI_data.columns.to_list()
columns=columns[1:]

In [48]:
# Tab 2 Map
def point_map(Variable='number_complaints', min_val=0, Identified_By='All'):
   
    hover = HoverTool(tooltips=[("Business Name(s)", "@business_name"),
                                ("Address", "@address"),
                                ("Active Licenses","@Active"),
                                ("Licensed Ever", "@License"),
                                ("No. Complaints","@number_complaints"),
                                ("No. Investigations", "@overall_num_investigations"),
                                ("No. Violations",'@total_violations')
                           ])
    
    global x
        
    if Identified_By == "All":
        x = df
    elif Identified_By == "License":
        x = df.loc[df["License"]]
    elif Identified_By == "Complaint Code":
        x = df.loc[df["License"]==False]
 
    point_map = CartoLight()*tract_LI_data.hvplot(c=Variable,
                     geo=True,
                     legend=True,
                     width=300,
                     height=200,
                     cmap='viridis',
                     frame_height=800,
                     frame_width=800,
                                  alpha = 0,
                                  line_alpha=.6,
                                  xaxis=None, yaxis=None,
                     hover_cols=['TRACT', 'Total Population','Active', 'Percent Not Hispanic, White',
                                'Percent Hispanic', 'Percent Not Hispanic, Black']
    )*x.loc[x[Variable]>=min_val].hvplot.points(c=Variable,  
                 title='All Entities with Licenses or Related Complaints', 
                 cmap='bmy',
                 xaxis=None, yaxis=None,
                 width=600,
                     height=400,
                 hover_cols=['business_name','address','Active','License',
                             'number_complaints','overall_num_investigations',
                            'total_violations']
                ).opts(tools=[hover])
    
    return point_map

variables=df.columns.to_list()
variables=variables[5:]

In [49]:
# Tab 2 table
def point_table(Variable='number_complaints', min_val=0, Identified_By='All'):
    
    pd.set_option('display.max_columns', None)
    
    x=df

    if Identified_By == "All":
        x = df
    elif Identified_By == "License":
        x = df.loc[df["License"]]
    elif Identified_By == "Complaint Code":
        x = df.loc[df["License"]==False]

    x = x.loc[x[Variable]>=min_val]
    
    y = pd.DataFrame(x).drop(["geometry"], axis=1)
    
    sio = StringIO()
    y.to_csv(sio)
    sio.seek(0)
    return sio

In [50]:
class TractExplorer(param.Parameterized):
    """A Panel dashboard class."""

    Variable = param.Selector(default='Active', objects=columns)
    
    Minimum_Population = param.Integer(1, bounds=(0, 10000))
    
    @param.depends('Variable','Minimum_Population') # This is a Python "decorator"!
    
    def make_map1(self):
        
        return tract_map(self.Variable, self.Minimum_Population)

In [51]:
class PointExplorer(param.Parameterized):
    """A Panel dashboard class."""

    Variable = param.Selector(default='Active', objects=variables)
    
    Minimum_Value = param.Integer(1)
    
    Identified_By = param.Selector(default='All', objects=["All","License","Complaint Code"])
    
    @param.depends('Variable', 'Minimum_Value', 'Identified_By')
    
    def make_map2(self):
        
        return point_map(self.Variable, self.Minimum_Value, self.Identified_By)
    
    def table2(self):
        
        return point_table(self.Variable, self.Minimum_Value, self.Identified_By).head()
    
    Download = pn.widgets.FileDownload(callback=point_table, filename='filtered_properties.csv')

In [52]:
mymap1 = TractExplorer()

tract_tab = pn.Row(mymap1.param, mymap1.make_map1)

In [53]:
mymap2 = PointExplorer()

tract_tab2 = pn.Row(
            pn.Param(mymap2.param, name='Settings (<a href="https://docs.google.com/spreadsheets/d/e/2PACX-1vQLqA58GBr8s335cEtvhoCl_Mb5l-Bwa2LkYK38RWOKYvbJAaCAAmtAd4M5zegiNWGdsq1cNmYakXwI/pubhtml?gid=0&single=true">click here for Data Dictionary</a>)', width=300, sizing_mode="fixed"), 
            pn.Column(mymap2.make_map2, "## Download Entity Data", mymap2.Download),
            min_height=1000)

In [54]:
info = "# About this Project"

In [55]:
tabs = pn.Tabs(('Tracts',tract_tab),("Entities",tract_tab2),('About', info))

In [56]:
tabs.servable()

Tabs
    [0] Row
        [0] Column(margin=5, name='TractExplorer', width=300)
            [0] StaticText(value='<b>TractExplorer</b>')
            [1] Select(name='Variable', options=OrderedDict([('Active', ...]), value='Active')
            [2] IntSlider(end=10000, name='Minimum Population', value=1)
        [1] ParamMethod(method)
    [1] Row(min_height=1000)
        [0] Param(PointExplorer, name='Settings (<a h..., sizing_mode='fixed', width=300)
        [1] Column
            [0] ParamMethod(method)
            [1] Markdown(str)
            [2] FileDownload(callback=<function point_table a..., filename='filtered_properties.csv', label='Download filtered_propert...)
    [2] Markdown(str, name='About')